# 使用所学的知识升级MNSIT网络
* 隐藏层激活函数：tanh
* 输出层激活函数：softmax
* 误差函数：交叉熵，（optional）
* alpha：0.02， epoch：300， 隐藏层节点：100个
* 批处理个数：100
* 选择前1000个数据进行训练

## 数据处理

In [1]:
import numpy as np
np.random.seed(1)

from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [2]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
x_train = x_train[0:1000]
x_train = x_train.reshape(x_train.shape[0], 28*28) / 255.0
y_train = y_train[0:1000]
x_test = x_test.reshape(x_test.shape[0], 28*28) / 255.0
# y_test = y_test[0:1000]

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1000, 784)
(1000,)
(10000, 784)
(10000,)


In [5]:
one_hot_labels = np.zeros((y_train.shape[0], 10))
print(y_train[0])
for i, num in enumerate(y_train):
    one_hot_labels[i][num] = 1
    pass
y_train = one_hot_labels
y_train[0]

5


array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [6]:
one_hot_labels = np.zeros((y_test.shape[0], 10))
print(y_test[0])
for i, num in enumerate(y_test):
    one_hot_labels[i][num] = 1
    pass
y_test = one_hot_labels
y_test[0]

7


array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

## 初始化网络参数

In [7]:
# 隐藏层激活函数：tanh
# 输出层激活函数：softmax
# 误差函数：交叉熵，（optional）
# alpha：0.02， epoch：300， 隐藏层节点：100个, 输出层节点：10个
# 批处理个数：100
# 选择前1000个数据进行训练

def tanh(x):
    return np.tanh(x)

def tanh2deriv(x):
    return 1 - np.power(x, 2)

def softmax(x):
    temp = np.exp(x)
    temp = temp / np.sum(temp, axis=1, keepdims=True)
    return temp

alpha = 0.01
epoch = 300
hidden_size = 100
output_size = 10
batch_size = 100
feature = x_train.shape[1]
np.random.seed(1)
weight_0_1 = 0.02 * np.random.random((feature, hidden_size)) - 0.01
weight_1_2 = 0.2 * np.random.random((hidden_size, output_size)) - 0.1
print(weight_0_1.shape)
print(weight_1_2.shape)
print(weight_0_1)

for i in range(epoch):
    correct = 0
    for j in range(int(x_train.shape[0] / batch_size)):
        batch_start = j * batch_size
        batch_end = (j+1) * batch_size
        label = y_train[batch_start: batch_end]
        # 前向传播
        layer_0 = x_train[batch_start: batch_end]
        layer_1 = np.dot(layer_0, weight_0_1)
        layer_1 = tanh(layer_1)
         # 关闭节点，防止过拟合
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        # * 为对应位置相乘并放大信号
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1, weight_1_2))
        
        for k in range(batch_size):
            correct += int(np.argmax(layer_2[k:k+1]) == np.argmax(label[k:k+1]))
            pass
        # 后向传播
        
        layer_2_delta = (label - layer_2) / batch_size
        layer_1_delta = np.dot(layer_2_delta, weight_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        
        weight_1_2 += alpha * np.dot(layer_1.T, layer_2_delta)
        weight_0_1 += alpha * np.dot(layer_0.T, layer_1_delta)
        
    print('i:',i, ' correct-rate: ', correct / 1000)



(784, 100)
(100, 10)
[[-0.00165956  0.00440649 -0.00999771 ...  0.00147359 -0.00994259
   0.0023429 ]
 [-0.0034671   0.00054116  0.00771884 ... -0.00531276  0.00233557
   0.00898033]
 [ 0.00900352  0.00113306  0.00831213 ... -0.00840748  0.00965634
  -0.00636774]
 ...
 [-0.00171772 -0.00421135 -0.00168973 ... -0.00358653  0.00238529
   0.00074803]
 [-0.00868448  0.00469209  0.00468847 ... -0.00826033 -0.0004682
   0.0014517 ]
 [ 0.00278132  0.00227828  0.00888591 ...  0.00668036 -0.00584658
   0.00298549]]
i: 0  correct-rate:  0.112
i: 1  correct-rate:  0.233
i: 2  correct-rate:  0.371
i: 3  correct-rate:  0.467
i: 4  correct-rate:  0.555
i: 5  correct-rate:  0.589
i: 6  correct-rate:  0.631
i: 7  correct-rate:  0.641
i: 8  correct-rate:  0.664
i: 9  correct-rate:  0.696
i: 10  correct-rate:  0.688
i: 11  correct-rate:  0.709
i: 12  correct-rate:  0.712
i: 13  correct-rate:  0.72
i: 14  correct-rate:  0.708
i: 15  correct-rate:  0.716
i: 16  correct-rate:  0.714
i: 17  correct-rate:  0

i: 272  correct-rate:  0.913
i: 273  correct-rate:  0.903
i: 274  correct-rate:  0.906
i: 275  correct-rate:  0.91
i: 276  correct-rate:  0.909
i: 277  correct-rate:  0.909
i: 278  correct-rate:  0.915
i: 279  correct-rate:  0.911
i: 280  correct-rate:  0.911
i: 281  correct-rate:  0.906
i: 282  correct-rate:  0.909
i: 283  correct-rate:  0.907
i: 284  correct-rate:  0.9
i: 285  correct-rate:  0.917
i: 286  correct-rate:  0.915
i: 287  correct-rate:  0.912
i: 288  correct-rate:  0.915
i: 289  correct-rate:  0.909
i: 290  correct-rate:  0.905
i: 291  correct-rate:  0.915
i: 292  correct-rate:  0.92
i: 293  correct-rate:  0.91
i: 294  correct-rate:  0.914
i: 295  correct-rate:  0.911
i: 296  correct-rate:  0.913
i: 297  correct-rate:  0.912
i: 298  correct-rate:  0.909
i: 299  correct-rate:  0.919


## 测试

In [8]:
correct = 0
for i in range(x_test.shape[0]):
        layer_0 = x_test[i: i+1]
        layer_1 = np.dot(layer_0, weight_0_1)
        layer_1 = tanh(layer_1)
        layer_2 = softmax(np.dot(layer_1, weight_1_2))
        correct += int(np.argmax(layer_2) == np.argmax(y_test[i:i+1]))
        print('i:',i, ' correct-rate: ', correct / y_test.shape[0])

i: 0  correct-rate:  0.0001
i: 1  correct-rate:  0.0002
i: 2  correct-rate:  0.0003
i: 3  correct-rate:  0.0004
i: 4  correct-rate:  0.0005
i: 5  correct-rate:  0.0006
i: 6  correct-rate:  0.0007
i: 7  correct-rate:  0.0008
i: 8  correct-rate:  0.0008
i: 9  correct-rate:  0.0009
i: 10  correct-rate:  0.001
i: 11  correct-rate:  0.001
i: 12  correct-rate:  0.0011
i: 13  correct-rate:  0.0012
i: 14  correct-rate:  0.0013
i: 15  correct-rate:  0.0014
i: 16  correct-rate:  0.0015
i: 17  correct-rate:  0.0016
i: 18  correct-rate:  0.0016
i: 19  correct-rate:  0.0017
i: 20  correct-rate:  0.0018
i: 21  correct-rate:  0.0019
i: 22  correct-rate:  0.002
i: 23  correct-rate:  0.0021
i: 24  correct-rate:  0.0022
i: 25  correct-rate:  0.0023
i: 26  correct-rate:  0.0024
i: 27  correct-rate:  0.0025
i: 28  correct-rate:  0.0026
i: 29  correct-rate:  0.0027
i: 30  correct-rate:  0.0028
i: 31  correct-rate:  0.0029
i: 32  correct-rate:  0.003
i: 33  correct-rate:  0.003
i: 34  correct-rate:  0.0031


i: 289  correct-rate:  0.0251
i: 290  correct-rate:  0.0251
i: 291  correct-rate:  0.0252
i: 292  correct-rate:  0.0253
i: 293  correct-rate:  0.0254
i: 294  correct-rate:  0.0255
i: 295  correct-rate:  0.0255
i: 296  correct-rate:  0.0256
i: 297  correct-rate:  0.0257
i: 298  correct-rate:  0.0258
i: 299  correct-rate:  0.0259
i: 300  correct-rate:  0.0259
i: 301  correct-rate:  0.026
i: 302  correct-rate:  0.0261
i: 303  correct-rate:  0.0262
i: 304  correct-rate:  0.0262
i: 305  correct-rate:  0.0263
i: 306  correct-rate:  0.0264
i: 307  correct-rate:  0.0265
i: 308  correct-rate:  0.0266
i: 309  correct-rate:  0.0267
i: 310  correct-rate:  0.0268
i: 311  correct-rate:  0.0269
i: 312  correct-rate:  0.027
i: 313  correct-rate:  0.027
i: 314  correct-rate:  0.0271
i: 315  correct-rate:  0.0271
i: 316  correct-rate:  0.0272
i: 317  correct-rate:  0.0272
i: 318  correct-rate:  0.0272
i: 319  correct-rate:  0.0273
i: 320  correct-rate:  0.0273
i: 321  correct-rate:  0.0273
i: 322  corre

i: 589  correct-rate:  0.0492
i: 590  correct-rate:  0.0493
i: 591  correct-rate:  0.0493
i: 592  correct-rate:  0.0494
i: 593  correct-rate:  0.0495
i: 594  correct-rate:  0.0496
i: 595  correct-rate:  0.0496
i: 596  correct-rate:  0.0497
i: 597  correct-rate:  0.0498
i: 598  correct-rate:  0.0499
i: 599  correct-rate:  0.05
i: 600  correct-rate:  0.0501
i: 601  correct-rate:  0.0502
i: 602  correct-rate:  0.0503
i: 603  correct-rate:  0.0504
i: 604  correct-rate:  0.0505
i: 605  correct-rate:  0.0506
i: 606  correct-rate:  0.0506
i: 607  correct-rate:  0.0507
i: 608  correct-rate:  0.0508
i: 609  correct-rate:  0.0509
i: 610  correct-rate:  0.0509
i: 611  correct-rate:  0.051
i: 612  correct-rate:  0.0511
i: 613  correct-rate:  0.0511
i: 614  correct-rate:  0.0512
i: 615  correct-rate:  0.0513
i: 616  correct-rate:  0.0514
i: 617  correct-rate:  0.0515
i: 618  correct-rate:  0.0516
i: 619  correct-rate:  0.0517
i: 620  correct-rate:  0.0518
i: 621  correct-rate:  0.0519
i: 622  corre

i: 912  correct-rate:  0.0761
i: 913  correct-rate:  0.0762
i: 914  correct-rate:  0.0763
i: 915  correct-rate:  0.0764
i: 916  correct-rate:  0.0765
i: 917  correct-rate:  0.0766
i: 918  correct-rate:  0.0767
i: 919  correct-rate:  0.0768
i: 920  correct-rate:  0.0769
i: 921  correct-rate:  0.077
i: 922  correct-rate:  0.0771
i: 923  correct-rate:  0.0772
i: 924  correct-rate:  0.0772
i: 925  correct-rate:  0.0773
i: 926  correct-rate:  0.0774
i: 927  correct-rate:  0.0775
i: 928  correct-rate:  0.0776
i: 929  correct-rate:  0.0777
i: 930  correct-rate:  0.0777
i: 931  correct-rate:  0.0778
i: 932  correct-rate:  0.0779
i: 933  correct-rate:  0.078
i: 934  correct-rate:  0.0781
i: 935  correct-rate:  0.0782
i: 936  correct-rate:  0.0782
i: 937  correct-rate:  0.0783
i: 938  correct-rate:  0.0783
i: 939  correct-rate:  0.0783
i: 940  correct-rate:  0.0784
i: 941  correct-rate:  0.0785
i: 942  correct-rate:  0.0786
i: 943  correct-rate:  0.0787
i: 944  correct-rate:  0.0787
i: 945  corr

i: 1191  correct-rate:  0.0989
i: 1192  correct-rate:  0.0989
i: 1193  correct-rate:  0.099
i: 1194  correct-rate:  0.0991
i: 1195  correct-rate:  0.0992
i: 1196  correct-rate:  0.0993
i: 1197  correct-rate:  0.0994
i: 1198  correct-rate:  0.0994
i: 1199  correct-rate:  0.0995
i: 1200  correct-rate:  0.0995
i: 1201  correct-rate:  0.0996
i: 1202  correct-rate:  0.0996
i: 1203  correct-rate:  0.0997
i: 1204  correct-rate:  0.0997
i: 1205  correct-rate:  0.0998
i: 1206  correct-rate:  0.0998
i: 1207  correct-rate:  0.0999
i: 1208  correct-rate:  0.0999
i: 1209  correct-rate:  0.0999
i: 1210  correct-rate:  0.1
i: 1211  correct-rate:  0.1001
i: 1212  correct-rate:  0.1002
i: 1213  correct-rate:  0.1003
i: 1214  correct-rate:  0.1004
i: 1215  correct-rate:  0.1005
i: 1216  correct-rate:  0.1006
i: 1217  correct-rate:  0.1006
i: 1218  correct-rate:  0.1007
i: 1219  correct-rate:  0.1007
i: 1220  correct-rate:  0.1008
i: 1221  correct-rate:  0.1008
i: 1222  correct-rate:  0.1009
i: 1223  cor

...

i: 8555  correct-rate:  0.7251
i: 8556  correct-rate:  0.7252
i: 8557  correct-rate:  0.7253
i: 8558  correct-rate:  0.7254
i: 8559  correct-rate:  0.7255
i: 8560  correct-rate:  0.7256
i: 8561  correct-rate:  0.7257
i: 8562  correct-rate:  0.7258
i: 8563  correct-rate:  0.7258
i: 8564  correct-rate:  0.7259
i: 8565  correct-rate:  0.726
i: 8566  correct-rate:  0.7261
i: 8567  correct-rate:  0.7262
i: 8568  correct-rate:  0.7263
i: 8569  correct-rate:  0.7264
i: 8570  correct-rate:  0.7265
i: 8571  correct-rate:  0.7266
i: 8572  correct-rate:  0.7267
i: 8573  correct-rate:  0.7268
i: 8574  correct-rate:  0.7269
i: 8575  correct-rate:  0.727
i: 8576  correct-rate:  0.7271
i: 8577  correct-rate:  0.7272
i: 8578  correct-rate:  0.7273
i: 8579  correct-rate:  0.7274
i: 8580  correct-rate:  0.7275
i: 8581  correct-rate:  0.7276
i: 8582  correct-rate:  0.7276
i: 8583  correct-rate:  0.7277
i: 8584  correct-rate:  0.7278
i: 8585  correct-rate:  0.7279
i: 8586  correct-rate:  0.728
i: 8587  co

i: 8867  correct-rate:  0.7551
i: 8868  correct-rate:  0.7552
i: 8869  correct-rate:  0.7553
i: 8870  correct-rate:  0.7554
i: 8871  correct-rate:  0.7555
i: 8872  correct-rate:  0.7556
i: 8873  correct-rate:  0.7557
i: 8874  correct-rate:  0.7558
i: 8875  correct-rate:  0.7559
i: 8876  correct-rate:  0.756
i: 8877  correct-rate:  0.7561
i: 8878  correct-rate:  0.7562
i: 8879  correct-rate:  0.7563
i: 8880  correct-rate:  0.7564
i: 8881  correct-rate:  0.7565
i: 8882  correct-rate:  0.7566
i: 8883  correct-rate:  0.7567
i: 8884  correct-rate:  0.7568
i: 8885  correct-rate:  0.7569
i: 8886  correct-rate:  0.757
i: 8887  correct-rate:  0.7571
i: 8888  correct-rate:  0.7572
i: 8889  correct-rate:  0.7573
i: 8890  correct-rate:  0.7574
i: 8891  correct-rate:  0.7575
i: 8892  correct-rate:  0.7576
i: 8893  correct-rate:  0.7577
i: 8894  correct-rate:  0.7578
i: 8895  correct-rate:  0.7579
i: 8896  correct-rate:  0.758
i: 8897  correct-rate:  0.7581
i: 8898  correct-rate:  0.7582
i: 8899  co

i: 9188  correct-rate:  0.7848
i: 9189  correct-rate:  0.7849
i: 9190  correct-rate:  0.785
i: 9191  correct-rate:  0.7851
i: 9192  correct-rate:  0.7852
i: 9193  correct-rate:  0.7853
i: 9194  correct-rate:  0.7854
i: 9195  correct-rate:  0.7855
i: 9196  correct-rate:  0.7856
i: 9197  correct-rate:  0.7857
i: 9198  correct-rate:  0.7858
i: 9199  correct-rate:  0.7859
i: 9200  correct-rate:  0.786
i: 9201  correct-rate:  0.7861
i: 9202  correct-rate:  0.7862
i: 9203  correct-rate:  0.7863
i: 9204  correct-rate:  0.7864
i: 9205  correct-rate:  0.7864
i: 9206  correct-rate:  0.7865
i: 9207  correct-rate:  0.7866
i: 9208  correct-rate:  0.7866
i: 9209  correct-rate:  0.7866
i: 9210  correct-rate:  0.7867
i: 9211  correct-rate:  0.7867
i: 9212  correct-rate:  0.7868
i: 9213  correct-rate:  0.7869
i: 9214  correct-rate:  0.787
i: 9215  correct-rate:  0.7871
i: 9216  correct-rate:  0.7872
i: 9217  correct-rate:  0.7873
i: 9218  correct-rate:  0.7874
i: 9219  correct-rate:  0.7875
i: 9220  co

i: 9550  correct-rate:  0.819
i: 9551  correct-rate:  0.8191
i: 9552  correct-rate:  0.8192
i: 9553  correct-rate:  0.8193
i: 9554  correct-rate:  0.8193
i: 9555  correct-rate:  0.8194
i: 9556  correct-rate:  0.8195
i: 9557  correct-rate:  0.8196
i: 9558  correct-rate:  0.8197
i: 9559  correct-rate:  0.8198
i: 9560  correct-rate:  0.8198
i: 9561  correct-rate:  0.8199
i: 9562  correct-rate:  0.82
i: 9563  correct-rate:  0.8201
i: 9564  correct-rate:  0.8201
i: 9565  correct-rate:  0.8202
i: 9566  correct-rate:  0.8203
i: 9567  correct-rate:  0.8204
i: 9568  correct-rate:  0.8205
i: 9569  correct-rate:  0.8206
i: 9570  correct-rate:  0.8207
i: 9571  correct-rate:  0.8208
i: 9572  correct-rate:  0.8209
i: 9573  correct-rate:  0.821
i: 9574  correct-rate:  0.8211
i: 9575  correct-rate:  0.8212
i: 9576  correct-rate:  0.8213
i: 9577  correct-rate:  0.8214
i: 9578  correct-rate:  0.8215
i: 9579  correct-rate:  0.8216
i: 9580  correct-rate:  0.8217
i: 9581  correct-rate:  0.8218
i: 9582  cor

i: 9848  correct-rate:  0.8439
i: 9849  correct-rate:  0.844
i: 9850  correct-rate:  0.8441
i: 9851  correct-rate:  0.8442
i: 9852  correct-rate:  0.8443
i: 9853  correct-rate:  0.8443
i: 9854  correct-rate:  0.8444
i: 9855  correct-rate:  0.8444
i: 9856  correct-rate:  0.8444
i: 9857  correct-rate:  0.8445
i: 9858  correct-rate:  0.8445
i: 9859  correct-rate:  0.8446
i: 9860  correct-rate:  0.8447
i: 9861  correct-rate:  0.8448
i: 9862  correct-rate:  0.8449
i: 9863  correct-rate:  0.845
i: 9864  correct-rate:  0.8451
i: 9865  correct-rate:  0.8452
i: 9866  correct-rate:  0.8453
i: 9867  correct-rate:  0.8453
i: 9868  correct-rate:  0.8454
i: 9869  correct-rate:  0.8455
i: 9870  correct-rate:  0.8455
i: 9871  correct-rate:  0.8456
i: 9872  correct-rate:  0.8457
i: 9873  correct-rate:  0.8458
i: 9874  correct-rate:  0.8458
i: 9875  correct-rate:  0.8458
i: 9876  correct-rate:  0.8459
i: 9877  correct-rate:  0.8459
i: 9878  correct-rate:  0.846
i: 9879  correct-rate:  0.846
i: 9880  cor